<a href="https://colab.research.google.com/github/Nicoxfl/Nicoxfl-MIT-Certificate-Data-Science/blob/main/Module_11_Regression_Assignment_11_1_Nicolas_Saiah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 33.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

LIST

In [ ]:
# Be careful with Python indices!
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,164))
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on...
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163]
157
158


PARAMETERS

In [ ]:
#From orginal price / sales(demand) data
# Developed linear demand model: used 27-156 weeks (3 years) of data; 27-104( years 1/2)Train, 105-156( last year)Test

# Variable: 𝑆𝑒𝑎𝑠𝑜𝑛X. Year divided int 13 seasons / 4 weeks per season (categorical variable)
# Variable: Price,  at time 𝑝_𝑡, 𝑝_(𝑡−1), 𝑝_(𝑡−2)
# Linear regression developed demand model (Constrain 1)
# Demand is characterized by our linearly additive model
# model: 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13
# model_new: 𝑑_𝑡 = 1.978242858 -2.809634145 *𝑝_𝑡 + 0.963410728*𝑝_(𝑡−1) + 0.759639170*𝑝_(𝑡−2) - 0.562046910*𝑆𝑒𝑎𝑠𝑜𝑛2 + 0.087545274*𝑆𝑒𝑎𝑠𝑜𝑛3 -0.402637480*𝑆𝑒𝑎𝑠𝑜𝑛4 -0.027326010*𝑆𝑒𝑎𝑠𝑜𝑛5 + 0.004349885*𝑆𝑒𝑎𝑠𝑜𝑛6 - 0.036102297*𝑆𝑒𝑎𝑠𝑜𝑛7 - 0.069280527*𝑆𝑒𝑎𝑠𝑜𝑛8 + 0.160276197*𝑆𝑒𝑎𝑠𝑜𝑛9 + 1.104208897*𝑆𝑒𝑎𝑠𝑜𝑛10 + 1.122711287*𝑆𝑒𝑎𝑠𝑜𝑛11 + 1.176802194*𝑆𝑒𝑎𝑠𝑜𝑛12 + 0.947945548*𝑆𝑒𝑎𝑠𝑜𝑛13


# Store model parameter_new
intercept = 1.978242858
# causal factors_new:
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
# time series factor
season_coeff = {1: 0, 2: -555.430, 3: 81.746,  4: -406.774, 5: -26.122, 6: -8.292,
               7: -39.334, 8: -81.407, 9: 148.728, 10: 1107.254, 11: 1125.259, 12: 1171.240, 13: 949.056}

In [ ]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: 1.0, 158: 1.0, 159: 1.0, 160: 1.0, 161: 2.0, 162: 2.0, 163: 2.0}



Model - no business rules + price ladder

In [ ]:
p_ladder = [1.0, 0.95, 0.9, 0.85, 0.75, 0.60, 0.50] #create a list for price ladder

In [ ]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [ ]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [ ]:
mod2.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 14 rows, 56 columns and 105 nonzeros
Model fingerprint: 0x7b2b528a
Model has 18 quadratic objective terms
Variable types: 7 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [2e+00, 6e+02]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -1215.806234
Presolve removed 0 rows and 7 columns
Presolve time: 0.00s
Presolved: 14 rows, 49 columns, 91 nonzeros
Presolved model has 18 quadratic objective terms
Variable types: 7 continuous, 42 integer (42 binary)

Root relaxation: objective -8.266156e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Object

In [ ]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x
df2

,price
157,1.0
158,1.0
159,0.95
160,0.85
161,0.5
162,0.5
163,0.5


In [ ]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()